In [1]:
# Get required libraries 
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

In [2]:
#Load train and test datasets and shuffle the data 
import time
start = time.clock()
train = shuffle(pd.read_csv("train.csv"))
test = shuffle(pd.read_csv("test.csv"))

In [6]:
# Seperating Predictors and Outcome values from train and test sets
X_train = pd.DataFrame(train.drop(['Activity','subject'],axis=1))
Y_train_label = train.Activity.values.astype(object)
X_test = pd.DataFrame(test.drop(['Activity','subject'],axis=1))
Y_test_label = test.Activity.values.astype(object)

# Dimension of Train and Test set 
print("Dimension of Train set",X_train.shape)
print("Dimension of Test set",X_test.shape,"\n")

# Transforming non numerical labels into numerical labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# encoding train labels 
encoder.fit(Y_train_label)
Y_train = encoder.transform(Y_train_label)

# encoding test labels 
encoder.fit(Y_test_label)
Y_test = encoder.transform(Y_test_label)

#Total Number of Continous and Categorical features in the training set
num_cols = X_train._get_numeric_data().columns
print("Number of numeric features:",num_cols.size)
#list(set(X_train.columns) - set(num_cols))


names_of_predictors = list(X_train.columns.values)

# Scaling the Train and Test feature set 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Dimension of Train set (7352, 561)
Dimension of Test set (2947, 561) 

Number of numeric features: 561


In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=200)
X_train_scaled = pca.fit_transform(X_train_scaled, Y_train)
print(pca.explained_variance_ratio_.sum())
X_test_scaled = pca.transform(X_test_scaled)

0.9936114684996993


In [8]:
X_test_scaled.shape

(2947, 200)

In [9]:
import tensorflow as tf
tf.__version__

/Users/randylee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'2.0.0'

### Tuning Neurons Number

In [10]:
def Create_model2(neurons1 = 1, neurons2 = 1, neurons3 = 1):
    
    #Create model
    model = tf.keras.models.Sequential()
    
    #First layer
    model.add(tf.keras.layers.Dense(neurons1, input_dim = 200,
                         kernel_initializer='uniform', activation='relu', ))
    model.add(tf.keras.layers.Dropout(0.1))
    
    #Hidden layer
    model.add(tf.keras.layers.Dense(neurons2, kernel_initializer='uniform', activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(neurons3, kernel_initializer='uniform', activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    
    #Output layer
    model.add(tf.keras.layers.Dense(6, kernel_initializer='uniform', activation='softmax'))
    
    #Compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#Create model
model = KerasClassifier(build_fn=Create_model2, epochs=100, batch_size=10, verbose=0)

#Define the grid search parameters
neurons1 = [48, 72, 96]
neurons2 = [24, 36, 48]
neurons3 = [12, 18, 24, 30]
param_grid = dict(neurons1=neurons1, neurons2=neurons2, neurons3=neurons3)
#param_grid = dict(neurons1=neurons1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
end = time.clock()
print('Running time: %s Seconds'%(end-start))

Using TensorFlow backend.


Best: 0.983950 using {'neurons1': 72, 'neurons2': 24, 'neurons3': 30}
0.980005 (0.004051) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 12}
0.982318 (0.000842) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 18}
0.979325 (0.004398) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 24}
0.980141 (0.001173) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 30}
0.981230 (0.001762) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 12}
0.980141 (0.001953) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 18}
0.981638 (0.002030) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 24}
0.982318 (0.000506) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 30}
0.980822 (0.004626) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 12}
0.980822 (0.001530) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 18}
0.980686 (0.003002) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 24}
0.980005 (0.001729) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 30}
0.982046 (0.001765) with: {'ne

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#Create model
model = KerasClassifier(build_fn=Create_model2, epochs=100, batch_size=10, verbose=0)

#Define the grid search parameters
neurons1 = [48]
neurons2 = [24]
neurons3 = [12]
param_grid = dict(neurons1=neurons1, neurons2=neurons2, neurons3=neurons3)
#param_grid = dict(neurons1=neurons1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
end = time.clock()
print('Running time: %s Seconds'%(end-start))

Using TensorFlow backend.


Best: 0.979325 using {'neurons1': 48, 'neurons2': 24, 'neurons3': 12}
0.979325 (0.002164) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 12}
Running time: 183.53139000000002 Seconds


In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#Create model
model = KerasClassifier(build_fn=Create_model2, epochs=100, batch_size=10, verbose=0)

#Define the grid search parameters
neurons1 = [48, 72, 96, 120, 144]
neurons2 = [24, 36, 48, 60, 72]
neurons3 = [12, 18, 24, 30, 36]
param_grid = dict(neurons1=neurons1, neurons2=neurons2, neurons3=neurons3)
#param_grid = dict(neurons1=neurons1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
end = time.clock()
print('Running time: %s Seconds'%(end-start))

Using TensorFlow backend.


Best: 0.986534 using {'neurons1': 48, 'neurons2': 36, 'neurons3': 30}
0.981638 (0.000663) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 12}
0.979597 (0.005039) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 18}
0.981230 (0.004256) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 24}
0.984766 (0.002217) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 30}
0.978373 (0.004703) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 36}
0.980686 (0.001018) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 12}
0.979733 (0.002453) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 18}
0.983270 (0.001336) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 24}
0.986534 (0.001204) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 30}
0.981910 (0.002361) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 36}
0.981774 (0.001680) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 12}
0.982862 (0.000579) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 18}
0.981910 (0.002412) with: {'ne

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#Create model
model = KerasClassifier(build_fn=Create_model2, epochs=10, batch_size=20, verbose=0)

#Define the grid search parameters
neurons1 = [48, 72, 96]
neurons2 = [24, 36, 48]
neurons3 = [12, 18, 24]
param_grid = dict(neurons1=neurons1, neurons2=neurons2, neurons3=neurons3)
#param_grid = dict(neurons1=neurons1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
end = time.clock()
print('Running time: %s Seconds'%(end-start))

Using TensorFlow backend.


Best: 0.977013 using {'neurons1': 96, 'neurons2': 48, 'neurons3': 18}
0.972933 (0.003886) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 12}
0.971436 (0.000574) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 18}
0.974021 (0.000510) with: {'neurons1': 48, 'neurons2': 24, 'neurons3': 24}
0.972388 (0.000839) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 12}
0.974837 (0.001919) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 18}
0.973885 (0.001449) with: {'neurons1': 48, 'neurons2': 36, 'neurons3': 24}
0.974565 (0.001013) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 12}
0.969668 (0.000775) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 18}
0.966268 (0.011963) with: {'neurons1': 48, 'neurons2': 48, 'neurons3': 24}
0.972797 (0.004136) with: {'neurons1': 72, 'neurons2': 24, 'neurons3': 12}
0.971844 (0.001860) with: {'neurons1': 72, 'neurons2': 24, 'neurons3': 18}
0.972797 (0.001507) with: {'neurons1': 72, 'neurons2': 24, 'neurons3': 24}
0.975517 (0.004327) with: {'ne